# Load Data

In [29]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.tree import export_graphviz
from six import StringIO
from IPython.display import Image  
import pydotplus
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import cohen_kappa_score, make_scorer
from sklearn import ensemble
import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.pipeline import Pipeline

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer



#SOURCE_DATA_FOLDER = "/kaggle/input/petfinder-adoption-prediction/"

#TRAIN_TABULAR = SOURCE_DATA_FOLDER + "train/train.csv"
#TEST_TABULAR = SOURCE_DATA_FOLDER + "test/test.csv"
#train_tabular_df = pd.read_csv(TRAIN_TABULAR)
#test_tabular_df = pd.read_csv(TEST_TABULAR)

kappa_scorer = make_scorer(cohen_kappa_score,weights='quadratic')

TRAIN_TABULAR = "data/train.csv"
BREED_LABELS =  "data/BreedLabels.csv"
COLOR_LABELS = "data/ColorLabels.csv"
STATE_LABELS = "data/StateLabels.csv"
TEST_TABULAR = "data/test.csv"

train_tabular_df = pd.read_csv(TRAIN_TABULAR)
test_tabular_df = pd.read_csv(TEST_TABULAR)
breed_labels_df = pd.read_csv(BREED_LABELS)
color_labels_df = pd.read_csv(COLOR_LABELS)
state_labels_df = pd.read_csv(STATE_LABELS)

In [30]:
train_tabular_temp = train_tabular_df.copy()
len_test = len(test_tabular_df)
len_train = len(train_tabular_df)
train_tabular_temp = train_tabular_temp.drop(columns=["AdoptionSpeed"])
adoptionspeed = train_tabular_df["AdoptionSpeed"]
frames = [train_tabular_temp, test_tabular_df]
  
result = pd.concat(frames)
display(result)

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt
0,2,Nibble,3,299,0,1,1,7,0,1,...,2,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,3,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0
2,1,Brisco,1,307,0,1,2,7,0,2,...,2,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0
3,1,Miko,4,307,0,2,1,2,0,2,...,2,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0
4,1,Hunter,1,307,0,1,1,0,0,2,...,2,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3967,1,Hugo,5,307,307,1,1,2,0,2,...,2,1,2,150,41401,18ec8ca4486bc2760de0bd5390cee30c,0,Found on the streets. Treated for mange. They ...,ae57f8d52,5.0
3968,1,Spot,30,307,307,1,1,2,7,2,...,2,1,1,0,41326,d83be5f5e2d04e24d7376e99eafd8708,0,Very good guard dog. Healthy was found in Fron...,83432904d,2.0
3969,2,NaN,5,300,0,3,1,2,4,2,...,2,1,6,0,41401,30aa45fdbe45e39d5614ef583b569073,0,these cat's mother was killed when they was ne...,399013029,1.0
3970,1,Smokey,24,307,0,2,5,7,0,2,...,2,1,1,0,41325,087903c2819a6297519c93d962b488d5,0,"smokey is good family pet. very obedient,so lo...",fd80b8c80,3.0


In [31]:
# create age bins
age_bins = pd.cut(result['Age'], bins=[0,3,6,12,24,300], include_lowest=True)
age_bin_dummies = pd.get_dummies(age_bins)
age_bin_dummies.columns = ["is_age_0_3", "is_age_3_6", "is_age_6_12","is_age_12_24", "is_age_24_300"]
result = pd.concat([result, age_bin_dummies], axis=1)

# one-hot encode dummy variables

dummy_cols = [
    'Type',
    'Gender',
    'MaturitySize',
    'Vaccinated',
    'Dewormed',
    'Sterilized',
    'Health',
    'FurLength',
    'State',
    'Breed1',
    'Breed2',
    'Color1',
    'Color2',
    'Color3',
]

result = pd.get_dummies(result, columns=dummy_cols)
result.head()

,Name,Age,Quantity,Fee,RescuerID,VideoAmt,Description,PetID,PhotoAmt,is_age_0_3,...,Color2_4,Color2_5,Color2_6,Color2_7,Color3_0,Color3_3,Color3_4,Color3_5,Color3_6,Color3_7
0,Nibble,3,1,100,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,1,...,0,0,0,1,1,0,0,0,0,0
1,No Name Yet,1,1,0,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,1,...,0,0,0,0,1,0,0,0,0,0
2,Brisco,1,1,0,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,1,...,0,0,0,1,1,0,0,0,0,0
3,Miko,4,1,150,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,0,...,0,0,0,0,1,0,0,0,0,0
4,Hunter,1,1,0,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,1,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3967,Hugo,5,2,150,18ec8ca4486bc2760de0bd5390cee30c,0,Found on the streets. Treated for mange. They ...,ae57f8d52,5.0,0,...,0,0,0,0,1,0,0,0,0,0
3968,Spot,30,1,0,d83be5f5e2d04e24d7376e99eafd8708,0,Very good guard dog. Healthy was found in Fron...,83432904d,2.0,0,...,0,0,0,0,0,0,0,0,0,1
3969,NaN,5,6,0,30aa45fdbe45e39d5614ef583b569073,0,these cat's mother was killed when they was ne...,399013029,1.0,0,...,0,0,0,0,0,0,1,0,0,0
3970,Smokey,24,1,0,087903c2819a6297519c93d962b488d5,0,"smokey is good family pet. very obedient,so lo...",fd80b8c80,3.0,0,...,0,0,0,1,1,0,0,0,0,0


In [32]:
train_transformed = result.head(len_train)
test_transformed = result.tail(len_test)
train_transformed = train_transformed.join(adoptionspeed)

rescuerid_counts = pd.DataFrame(train_transformed.groupby(["RescuerID"]).size(), columns=["RescuerID_Count"]).reset_index()
train_transformed = pd.merge(train_transformed, rescuerid_counts, on="RescuerID")
rescuerid_counts = pd.DataFrame(test_transformed.groupby(["RescuerID"]).size(), columns=["RescuerID_Count"]).reset_index()
test_transformed = pd.merge(test_transformed, rescuerid_counts, on="RescuerID")

# Preprocessing data and transform variables

In [35]:
train_transformed["Name"] = train_transformed["Name"].fillna(0)
train_transformed.loc[train_transformed.Name != 0, "Name"] = 1
train_transformed.drop(columns=['RescuerID','Description','Fee','PetID'],inplace=True)
train_transformed["Name"] = train_transformed["Name"].astype(str).astype(float)

,Name,Age,Quantity,VideoAmt,PhotoAmt,is_age_0_3,is_age_3_6,is_age_6_12,is_age_12_24,is_age_24_300,...,Color2_6,Color2_7,Color3_0,Color3_3,Color3_4,Color3_5,Color3_6,Color3_7,AdoptionSpeed,RescuerID_Count
0,1.0,3,1,0,1.0,1,0,0,0,0,...,0,1,1,0,0,0,0,0,2,8
1,1.0,12,1,0,1.0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,2,8
2,1.0,3,1,0,1.0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,2,8
3,1.0,4,2,0,2.0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,3,8
4,1.0,7,1,0,4.0,0,0,1,0,0,...,0,1,1,0,0,0,0,0,2,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14988,1.0,2,1,0,0.0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,1
14989,1.0,84,2,0,3.0,0,0,0,0,1,...,0,1,1,0,0,0,0,0,3,1
14990,1.0,3,1,0,2.0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,4,1
14991,1.0,24,1,0,1.0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,4,1


In [36]:
test_transformed['Name'] = test_transformed['Name'].fillna(0)
test_transformed.loc[test_transformed.Name != 0, "Name"] = 1
test_transformed.drop(columns=['RescuerID','Description','Fee'],inplace=True)
test_transformed["Name"] = test_transformed["Name"].astype(str).astype(float)

,Name,Age,Quantity,VideoAmt,PetID,PhotoAmt,is_age_0_3,is_age_3_6,is_age_6_12,is_age_12_24,...,Color2_5,Color2_6,Color2_7,Color3_0,Color3_3,Color3_4,Color3_5,Color3_6,Color3_7,RescuerID_Count
0,1.0,8,2,0,e2dfc2935,2.0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,2
1,1.0,36,1,0,f153b465f,1.0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2
2,1.0,2,1,0,3c90f3f54,4.0,1,0,0,0,...,0,0,1,1,0,0,0,0,0,6
3,1.0,12,2,0,e02abc8a3,5.0,0,0,1,0,...,1,0,0,1,0,0,0,0,0,6
4,1.0,12,1,0,09f0df7d1,5.0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3967,1.0,5,2,0,ae57f8d52,5.0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1
3968,1.0,30,1,0,83432904d,2.0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
3969,0.0,5,6,0,399013029,1.0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,1
3970,1.0,24,1,0,fd80b8c80,3.0,0,0,0,1,...,0,0,1,1,0,0,0,0,0,1


In [37]:
Y_tr = train_transformed["AdoptionSpeed"]
X_tr = train_transformed.drop(columns=["AdoptionSpeed"])
test = test_transformed.drop(columns=["PetID"])

In [44]:
clf_xgb = XGBClassifier(objective='multi:softmax',colsample_bytree = 0.5, learning_rate = 0.06, max_depth = 5, min_child_weight = 1, n_estimators = 200, subsample = 0.7,num_class=5,use_label_encoder=False,eval_metric='merror')
clf_xgb.fit(X_tr,Y_tr)
print("Training score:", cohen_kappa_score(clf_xgb.predict(X_tr), Y_tr, weights='quadratic'))

Training score: 0.45268100094270247


In [45]:
predicted = clf_xgb.predict(test)

In [46]:
submission = pd.DataFrame.from_dict({'PetID': test_transformed['PetID'],
                                     'AdoptionSpeed': predicted})
print(submission.shape)
submission

(3972, 2)


,PetID,AdoptionSpeed
0,e2dfc2935,4
1,f153b465f,4
2,3c90f3f54,1
3,e02abc8a3,4
4,09f0df7d1,4
...,...,...
3967,ae57f8d52,4
3968,83432904d,4
3969,399013029,4
3970,fd80b8c80,4
